In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import gradio as gr
from pypdf import PdfReader

In [ ]:
# load environment variables from .env file
load_dotenv(override=True)

# load OpenAI API key and base URL from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_base_url = os.getenv("OPENAI_API_BASE")
openai_default_model = os.getenv("OPENAI_DEFAULT_MODEL")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in environment variables.")

# initialize OpenAI client
openai = OpenAI(api_key=openai_api_key)

if openai_base_url:
    openai.base_url = openai_base_url

In [ ]:
# extract information from resume PDF
reader = PdfReader("resources/resume.pdf")
reseume_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        reseume_text += text

In [ ]:
reseume_text

In [ ]:
summary_text = ""
with open("resources/summary.txt", "r", encoding="utf-8") as f:
    summary_text = f.read()

In [ ]:
name = "Asir Mosaddek Sakib"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary_text}\n\n## Resume:\n{reseume_text}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
system_prompt

In [ ]:
def chat(message, chat_history):
    # build messages for the OpenAI chat API using the stored chat_history (list of (user, bot) tuples)
    history = [{"role": h["role"], "content": h["content"]} for h in chat_history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(
        model=openai_default_model if openai_default_model else "gpt-4.1-mini",
        messages=messages
    )
    # note: .choices is the correct attribute name
    content = response.choices[0].message.content
    return content


In [ ]:
demo = gr.ChatInterface(fn=chat, title="Asir's Chatbot")
demo.launch(share = True)